<a href="https://colab.research.google.com/github/arnabsaha7/SMS-Spam-Detection-Model/blob/main/SMS_Spam_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Overview**
# **Code by: Arnab Saha - Bharat Intern**
## **Task 1:** Develop a text classification model to classify SMS as either spam or non-spam using data science techniques in Python
## **Dataset:** kaggle.com

# Approach


1.   Data initialization & Basic pre-processing
2.   Processing the Messages
3.   Classification and Splitting the Dataset
4.   Applying Gaussian Naive Bayes
5.   Applying Decision Tree
6.   Final resut





# Import Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [2]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Data Preprocessing

In [3]:
df = pd.read_csv('/content/spam.csv', encoding='latin-1')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [5]:
df = df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1)

In [6]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [7]:
# Replace ham with 0 and spam with 1
df = df.replace(['ham', 'spam'],[0, 1])

In [8]:
df.head()

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [9]:
# Add a column containing word counts of texts
df['Count']=0
for i in np.arange(0,len(df.v2)):
  df.loc[i, 'Count'] = len(df.loc[i,'v2'])

In [10]:
df.head()

,v1,v2,Count
0,0,"Go until jurong point, crazy.. Available only ...",111
1,0,Ok lar... Joking wif u oni...,29
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,0,U dun say so early hor... U c already then say...,49
4,0,"Nah I don't think he goes to usf, he lives aro...",61


In [11]:
# Total non-spam and spam count
df['v1'].value_counts()

0    4825
1     747
Name: v1, dtype: int64

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   int64 
 1   v2      5572 non-null   object
 2   Count   5572 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 130.7+ KB


In [13]:
corpus = []
ps = PorterStemmer()

In [14]:
print(df['v2'][0])
print(df['v2'][1])

Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
Ok lar... Joking wif u oni...


# Processing Texts

In [15]:
# Applying Regular Expression
for i in range(0, 5572):

  '''
    Replace email address with 'emailaddr'
    Replace URLs with 'httpaddr'
    Replace money symbols with 'moneysymb'
    Replace phone no. with 'phonenumbr'
    Replace numbers with 'numbr'
  '''
  msg = df['v2'][i]
  msg = re.sub('\b[\w\-.]+?@\w+?\.\w{2,4}\b', 'emailaddr', df['v2'][i])
  msg = re.sub('(http[s]?\S+)|(\w+\.[A-Za-z]{2,4}\S*)', 'httpaddr', df['v2'][i])
  msg = re.sub('£|\$', 'moneysymb', df['v2'][i])
  msg = re.sub('\b(\+\d{1,2}\s)?\d?[\-(.]?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}\b', 'phonenumbr', df['v2'][i])
  msg = re.sub('\d+(\.\d+)?', 'numbr', df['v2'][i])

  # Remove all punctuations
  msg = re.sub('[^\w\d\s]', ' ',df['v2'][i])

  if i<2:
      print("\t\t\t\t MESSAGE ", i)

  if i<2:
      print("\n After Regular Expression - Message ", i, " : ", msg)

  # Each word to lower case
  msg = msg.lower()
  if i<2:
      print("\n Lower case Message ", i, " : ", msg)

  # Splitting words to Tokenize
  msg = msg.split()
  if i<2:
      print("\n After Splitting - Message ", i, " : ", msg)

  # Stemming with PorterStemmer handling Stop Words
  msg = [ps.stem(word) for word in msg if not word in set(stopwords.words('english'))]
  if i<2:
      print("\n After Stemming - Message ", i, " : ", msg)

  # preparing Messages with Remaining Tokens
  msg = ' '.join(msg)
  if i<2:
      print("\n Final Prepared - Message ", i, " : ", msg, "\n\n")

  # Preparing WordVector Corpus
  corpus.append(msg)

				 MESSAGE  0

 After Regular Expression - Message  0  :  Go until jurong point  crazy   Available only in bugis n great world la e buffet    Cine there got amore wat   

 Lower case Message  0  :  go until jurong point  crazy   available only in bugis n great world la e buffet    cine there got amore wat   

 After Splitting - Message  0  :  ['go', 'until', 'jurong', 'point', 'crazy', 'available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'there', 'got', 'amore', 'wat']

 After Stemming - Message  0  :  ['go', 'jurong', 'point', 'crazi', 'avail', 'bugi', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'got', 'amor', 'wat']

 Final Prepared - Message  0  :  go jurong point crazi avail bugi n great world la e buffet cine got amor wat 


				 MESSAGE  1

 After Regular Expression - Message  1  :  Ok lar    Joking wif u oni   

 Lower case Message  1  :  ok lar    joking wif u oni   

 After Splitting - Message  1  :  ['ok', 'lar', 'joking', 'wif', '

In [16]:
cv = CountVectorizer()
x = cv.fit_transform(corpus).toarray()

# Classification


1.   Input: Prepared Sparse Matrix
2.   Output: Labels



In [17]:
y = df['v1']
print(y.value_counts())

print(y[0])
print(y[1])

0    4825
1     747
Name: v1, dtype: int64
0
0


**Encoding Labels**

In [18]:
le = LabelEncoder()
y = le.fit_transform(y)

print(y[0])
print(y[1])

0
0


### Train-Test Split

In [19]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=0)

# Applying Gaussian Naive Bayes

In [20]:
model = GaussianNB()
model.fit(xtrain, ytrain)

GaussianNB()

### Making Predictions

In [21]:
y_pred = model.predict(xtest)

### Result

In [23]:
cm = confusion_matrix(ytest, y_pred)
cm

array([[824, 125],
       [ 19, 147]])

In [25]:
print("Accuracy: %0.5f \n\n" % accuracy_score(ytest,model.predict(xtest)))
print(classification_report(ytest, model.predict(xtest)))

Accuracy: 0.87085 


              precision    recall  f1-score   support

           0       0.98      0.87      0.92       949
           1       0.54      0.89      0.67       166

    accuracy                           0.87      1115
   macro avg       0.76      0.88      0.80      1115
weighted avg       0.91      0.87      0.88      1115



# Applying Decision Tree

In [26]:
dt = DecisionTreeClassifier(random_state=42)
dt.fit(xtrain, ytrain)

DecisionTreeClassifier(random_state=42)

## Predictions

In [27]:
y_pred_dt = dt.predict(xtest)

## Result

In [28]:
cm = confusion_matrix(ytest, y_pred_dt)
print(cm)

[[942   7]
 [ 30 136]]


In [29]:
print ("Accuracy : %0.5f \n\n" % accuracy_score(ytest, dt.predict(xtest)))
print (classification_report(ytest, dt.predict(xtest)))

Accuracy : 0.96682 


              precision    recall  f1-score   support

           0       0.97      0.99      0.98       949
           1       0.95      0.82      0.88       166

    accuracy                           0.97      1115
   macro avg       0.96      0.91      0.93      1115
weighted avg       0.97      0.97      0.97      1115



# **FINAL ACCURACY SCORE**

*   **Decision Tree: 96.682%**
*   **Gaussian NB: 87.085%**

